In [1]:
import numpy as np
import pandas as pd

In [2]:
STable1 = pd.read_csv("STable1_all_genomes_info.tsv", sep="|")
STable2 = pd.read_csv("STable2_BiG-SLICE_t0.4_GCF_assignment.csv", sep=",")
STable3 = pd.read_csv("STable3_BiG-SLICE_t0.5_GCF_assignment.csv", sep=",")
STable4 = pd.read_csv("STable4_BiG-SLICE_t0.6_GCF_assignment.csv", sep=",")
STable5 = pd.read_csv("STable5_BiG-SLICE_t0.7_GCF_assignment.csv", sep=",")

In [104]:
new_table1_taxonomy = STable1.taxonomy.str.split(",",expand=True)

In [110]:
STable1

,dataset_name,AccNo,taxonomy,bgc_ids
0,mag_uba,DBAD,"Bacteria,Proteobacteria,Gammaproteobacteria,Bu...","1201186,1201187,1201188,1201189"
1,mag_uba,DBAF,"Bacteria,Proteobacteria,Gammaproteobacteria,Ps...","1201638,1201639,1201640,1201641,1201642,1201643"
2,mag_uba,DBAG,"Bacteria,Proteobacteria,Gammaproteobacteria,Ne...","1202199,1202200,1202201,1202202"
3,mag_uba,DBAH,"Bacteria,Bacteroidota,Bacteroidia,Flavobacteri...","1202184,1202185"
4,mag_uba,DBAI,"Bacteria,SAR324,NAC60-12,UBA1014,UBA1014 sp002...",1204170
...,...,...,...,...
217642,GEMS,3300029947_3,"Bacteria,Bacteroidota,Bacteroidia,Bacteroidale...",1293123
217643,GEMS,3300029947_7,"Bacteria,Bacteroidota,Bacteroidia,Bacteroidale...",1266441
217644,GEMS,3300029947_9,"Bacteria,Bacteroidota,Bacteroidia,Bacteroidale...",1283834
217645,GEMS,3300029948_2,"Bacteria,Bacteroidota,Bacteroidia,Flavobacteri...",1240339


In [122]:
new_table1_taxonomy[6].str.startswith("Strepto").dropna()

0         False
1         False
2         False
3         False
5         False
          ...  
217642    False
217643    False
217644    False
217645    False
217646    False
Name: 6, Length: 205819, dtype: object

In [127]:
new_table1_taxonomy[new_table1_taxonomy[6].isna()] = "Empty"

In [130]:
new_table1_taxonomy[new_table1_taxonomy[5].str.startswith("Streptomyces")]

,0,1,2,3,4,5,6
5494,Bacteria,Actinobacteriota,Actinobacteria,Streptomycetales,Streptomycetaceae,Streptomyces,Streptomyces avermitilis
5524,Bacteria,Actinobacteriota,Actinobacteria,Streptomycetales,Streptomycetaceae,Streptomyces,Streptomyces griseus
6190,Bacteria,Actinobacteriota,Actinobacteria,Streptomycetales,Streptomycetaceae,Streptomyces,Streptomyces scabiei
6209,Bacteria,Actinobacteriota,Actinobacteria,Streptomycetales,Streptomycetaceae,Streptomyces,Streptomyces milbemycinicus
6377,Bacteria,Actinobacteriota,Actinobacteria,Streptomycetales,Streptomycetaceae,Streptomyces,Streptomyces violaceusniger
...,...,...,...,...,...,...,...
175570,Bacteria,Actinobacteriota,Actinobacteria,Streptomycetales,Streptomycetaceae,Streptomyces,Streptomyces sp000500635
176757,Bacteria,Actinobacteriota,Actinobacteria,Streptomycetales,Streptomycetaceae,Streptomyces,Streptomyces albus
185216,Bacteria,Actinobacteriota,Actinobacteria,Streptomycetales,Streptomycetaceae,Streptomyces,UBA8490
194217,Bacteria,Actinobacteriota,Actinobacteria,Streptomycetales,Streptomycetaceae,Streptomyces_B,


In [115]:
new_table1_taxonomy

,0,1,2,3,4,5,6
0,Bacteria,Proteobacteria,Gammaproteobacteria,Burkholderiales,Burkholderiaceae,Ralstonia,Ralstonia sp900115545
1,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Pseudomonas_D,Pseudomonas_D pachastrellae
2,Bacteria,Proteobacteria,Gammaproteobacteria,Nevskiales,Salinisphaeraceae,Salinisphaera,Salinisphaera sp002715985
3,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Flavobacteriaceae,Zunongwangia,Zunongwangia profunda
4,Bacteria,SAR324,NAC60-12,UBA1014,UBA1014 sp002683785,None,None
...,...,...,...,...,...,...,...
217642,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Bacteroidaceae,Prevotella,GCA_002298545.1
217643,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Bacteroidaceae,Prevotella,Prevotella sp11
217644,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Paludibacteraceae,RF16,
217645,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Flavobacteriaceae,BACL21,GCA_002440705.1


In [111]:
STable1[["taxonomy","bgc_ids"]]

,taxonomy,bgc_ids
0,"Bacteria,Proteobacteria,Gammaproteobacteria,Bu...","1201186,1201187,1201188,1201189"
1,"Bacteria,Proteobacteria,Gammaproteobacteria,Ps...","1201638,1201639,1201640,1201641,1201642,1201643"
2,"Bacteria,Proteobacteria,Gammaproteobacteria,Ne...","1202199,1202200,1202201,1202202"
3,"Bacteria,Bacteroidota,Bacteroidia,Flavobacteri...","1202184,1202185"
4,"Bacteria,SAR324,NAC60-12,UBA1014,UBA1014 sp002...",1204170
...,...,...
217642,"Bacteria,Bacteroidota,Bacteroidia,Bacteroidale...",1293123
217643,"Bacteria,Bacteroidota,Bacteroidia,Bacteroidale...",1266441
217644,"Bacteria,Bacteroidota,Bacteroidia,Bacteroidale...",1283834
217645,"Bacteria,Bacteroidota,Bacteroidia,Flavobacteri...",1240339


In [131]:
new_list

0          27982
1          46412
2          39690
3          52360
4          20486
           ...  
1185990    15999
1185991      326
1185992    24789
1185993      137
1185994    24351
Name: gcf_id, Length: 1185995, dtype: int64

In [ ]:
count = 0

for i in new_list:
    new_new = newdf.isin([str(i)]).value_counts()
    if len(new_new.values) == 2:
        count = count + 1
    else:
        continue

In [15]:
bgc_ids = STable1.bgc_ids.str.split(",",expand=True)

In [17]:
bgc_ids[["dataset_name","AccNo","taxonomy"]] = STable1[["dataset_name","AccNo","taxonomy"]]

In [58]:
bgc_ids.isin([4027]).value_counts().index[0]

(False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False)

In [ ]:
pd.DataFrame.isin

In [51]:
bgc_ids[107].value_counts()

107
368701    1
Name: count, dtype: int64

In [39]:
bgc_ids.where(bgc_ids==4027,False)

,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,dataset_name,AccNo,taxonomy
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217642,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
217643,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
217644,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
217645,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [63]:
STable1.bgc_ids[0].split(",")

['1201186', '1201187', '1201188', '1201189']

In [64]:
"1250".split(",")

['1250']

In [62]:
STable2.bgc_id[0]

4023

In [66]:
all_bgcs = []
for i in range(len(STable1)):
    all_bgcs.extend(STable1.bgc_ids[i].split(","))

In [68]:
len(all_bgcs)

1185995

In [70]:
all_bgcs.sort()

Here the sorted all_bgcs from STABLE_1 we do not have "4027" bgc_id from STABLE_2, So this tells we have some missing bgc id in STABLE_1, this 

In [75]:
all_bgcs.index(4027)

ValueError: 4027 is not in list

In [74]:
all_bgcs.index(1317464)

ValueError: 1317464 is not in list

In [76]:
newdf = pd.DataFrame({"all_bgcs":all_bgcs})

In [80]:
newdf.isin(list(STable2.gcf_id)).value_counts()

all_bgcs
False       1185995
Name: count, dtype: int64

In [82]:
newdf

,all_bgcs
0,10000
1,100000
2,1000000
3,1000001
4,1000002
...,...
1185990,999995
1185991,999996
1185992,999997
1185993,999998


In [84]:
new_list

0          27982
1          46412
2          39690
3          52360
4          20486
           ...  
1185990    15999
1185991      326
1185992    24789
1185993      137
1185994    24351
Name: gcf_id, Length: 1185995, dtype: int64

In [89]:
newdf

,all_bgcs
0,10000
1,100000
2,1000000
3,1000001
4,1000002
...,...
1185990,999995
1185991,999996
1185992,999997
1185993,999998


In [94]:
newdf.isin(["10000"]).value_counts().values

array([1185994,       1])

In [83]:
new_list = STable2.gcf_id

In [97]:
type(new_list[0])

numpy.int64

In [98]:
i = 10000
newdf.isin([str(i)])

,all_bgcs
0,True
1,False
2,False
3,False
4,False
...,...
1185990,False
1185991,False
1185992,False
1185993,False


In [112]:
# count = 0

# for i in new_list:
#     new_new = newdf.isin([str(i)]).value_counts()
#     if len(new_new.values) == 2:
#         count = count + 1
#     else:
#         continue

In [100]:
count

28193

In [73]:
STable1

,dataset_name,AccNo,taxonomy,bgc_ids
0,mag_uba,DBAD,"Bacteria,Proteobacteria,Gammaproteobacteria,Bu...","1201186,1201187,1201188,1201189"
1,mag_uba,DBAF,"Bacteria,Proteobacteria,Gammaproteobacteria,Ps...","1201638,1201639,1201640,1201641,1201642,1201643"
2,mag_uba,DBAG,"Bacteria,Proteobacteria,Gammaproteobacteria,Ne...","1202199,1202200,1202201,1202202"
3,mag_uba,DBAH,"Bacteria,Bacteroidota,Bacteroidia,Flavobacteri...","1202184,1202185"
4,mag_uba,DBAI,"Bacteria,SAR324,NAC60-12,UBA1014,UBA1014 sp002...",1204170
...,...,...,...,...
217642,GEMS,3300029947_3,"Bacteria,Bacteroidota,Bacteroidia,Bacteroidale...",1293123
217643,GEMS,3300029947_7,"Bacteria,Bacteroidota,Bacteroidia,Bacteroidale...",1266441
217644,GEMS,3300029947_9,"Bacteria,Bacteroidota,Bacteroidia,Bacteroidale...",1283834
217645,GEMS,3300029948_2,"Bacteria,Bacteroidota,Bacteroidia,Flavobacteri...",1240339


In [132]:
STable2[STable2.gcf_id == 20486]

,bgc_id,gcf_id,feature_to_centroid_distance
4,4027,20486,0.184566
19,4042,20486,0.481559
27,4050,20486,0.246591
36,4059,20486,0.219628
54,4077,20486,0.177339
...,...,...,...
1184726,1317248,20486,0.405103
1184938,1317464,20486,0.293406
1185549,1318094,20486,0.451711
1185685,1318232,20486,0.403270


In [137]:
my_gcf_ids_stable2 = STable2.gcf_id.value_counts().values

In [138]:
my_gcf_ids_stable2[0]

32685

In [142]:
bgc_ids

,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,dataset_name,AccNo,taxonomy
0,1201186,1201187,1201188,1201189,None,None,None,None,None,None,...,None,None,None,None,None,None,None,mag_uba,DBAD,"Bacteria,Proteobacteria,Gammaproteobacteria,Bu..."
1,1201638,1201639,1201640,1201641,1201642,1201643,None,None,None,None,...,None,None,None,None,None,None,None,mag_uba,DBAF,"Bacteria,Proteobacteria,Gammaproteobacteria,Ps..."
2,1202199,1202200,1202201,1202202,None,None,None,None,None,None,...,None,None,None,None,None,None,None,mag_uba,DBAG,"Bacteria,Proteobacteria,Gammaproteobacteria,Ne..."
3,1202184,1202185,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,mag_uba,DBAH,"Bacteria,Bacteroidota,Bacteroidia,Flavobacteri..."
4,1204170,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,mag_uba,DBAI,"Bacteria,SAR324,NAC60-12,UBA1014,UBA1014 sp002..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217642,1293123,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,GEMS,3300029947_3,"Bacteria,Bacteroidota,Bacteroidia,Bacteroidale..."
217643,1266441,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,GEMS,3300029947_7,"Bacteria,Bacteroidota,Bacteroidia,Bacteroidale..."
217644,1283834,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,GEMS,3300029947_9,"Bacteria,Bacteroidota,Bacteroidia,Bacteroidale..."
217645,1240339,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,GEMS,3300029948_2,"Bacteria,Bacteroidota,Bacteroidia,Flavobacteri..."


In [164]:
bgc_ids[bgc_ids.isin(["1201186"])].isnull()

,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,dataset_name,AccNo,taxonomy
0,False,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217642,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
217643,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
217644,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
217645,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [150]:
mydf_agg = list(STable2[STable2.gcf_id == 12013].bgc_id)

In [152]:
mydf_agg = [str(all) for all in mydf_agg]

In [153]:
mydf_agg

['63826']

In [145]:
my_gcf_ids_stable2

array([32685, 12013, 11884, ...,     1,     1,     1])

In [ ]:
for i in range(len(my_gcf_ids_stable2)):
    my_id = my_gcf_ids_stable2[i]
    mydf_agg = list(STable2[STable2.gcf_id == my_id].bgc_id)
    mydf_agg = [str(all) for all in mydf_agg]
    bgc_ids.isin(mydf_agg)




    

In [21]:
STable2.bgc_id.value_counts()

bgc_id
4023       1
794884     1
794936     1
794935     1
794934     1
          ..
399406     1
399405     1
399404     1
399403     1
1318552    1
Name: count, Length: 1185995, dtype: int64

In [5]:
STable3

,bgc_id,gcf_id,feature_to_centroid_distance
0,4023,13160,0.501575
1,4024,34246,0.000000
2,4025,29260,0.038236
3,4026,38999,0.331269
4,4027,7025,0.205082
...,...,...,...
1060587,1064956,18781,0.195304
1060588,1064957,37887,0.353260
1060589,1064958,4387,0.363425
1060590,1064959,40957,0.212490


In [6]:
STable4

,bgc_id,gcf_id,feature_to_centroid_distance
0,4023,10044,0.644872
1,4024,25597,0.000000
2,4025,21492,0.038236
3,4026,26864,0.306828
4,4027,1874,0.205468
...,...,...,...
1060587,1064956,13335,0.195304
1060588,1064957,28533,0.353260
1060589,1064958,2899,0.365717
1060590,1064959,30430,0.214389


In [7]:
STable5

,bgc_id,gcf_id,feature_to_centroid_distance
0,4023,2585,0.641722
1,4024,3589,0.817356
2,4025,14552,0.038236
3,4026,18953,0.306828
4,4027,1706,0.205582
...,...,...,...
1060587,1064956,8094,0.260165
1060588,1064957,20620,0.359439
1060589,1064958,1888,0.364628
1060590,1064959,6524,0.644446
